### 1. Reset the reference channel / threshold
1. Record open-phys files by clicking the "MUA fifo" button (just for several seconds to generate files)
2. Run the codes below to reset reference and threshold
3. Delete bin files (`rm *.bin`)
4. Proceed to the next section

In [ ]:
%matplotlib inline
%gui qt

import numpy as np
from spiketag.mvc import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32.prb'

prb = probe()
prb.load(prb_filename)

ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename,
                  time_segs=[1, 2]
                 )

# reset reference channel
ch_ref_array = np.array(160*[160])
ctrl.fpga.set_channel_ref(ch_ref_array)

thres = np.ones(160) * -20000.0

thres[:64] = -200.0
thres[64:128] = -200.0 # -20000.0 if not used

# reset threshold level
ctrl.fpga.set_threshold(thres)

In [ ]:
print(ctrl.fpga.thres)
print(ctrl.fpga.ch_ref)

### 2. Select reference electrodes
1. Record for 1 minute (using open-ephys)
2. Run the following command to check the spike count per channel. Choose silent channels as the references per probe:
```bash
> spiketag report mua.bin spk.bin prb_a2x32.prb
```

3. Set the reference channel
4. Remove bin files (`rm *.bin`)
5. Proceed to the next section


In [ ]:
%matplotlib inline
%gui qt

import numpy as np
from spiketag.mvc import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32.prb'

prb = probe()
prb.load(prb_filename)
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                 )

# set reference channel
ch_ref_array = np.array(160*[160])

ref_chNo_0 = 38 # change this value
ref_chNo_1 = 160 # change this value

ch_ref_array[prb.chs[:64]] = ref_chNo_0
ch_ref_array[prb.chs[64:128]] = ref_chNo_1

ctrl.fpga.set_channel_ref(ch_ref_array)

### 3. Set threshold
1. Record for 1 minute
2. Set threshold
3. Remove bin files
4. Record about 10 minutes for clustering
5. Copy `sorter-fpga.ipynb` by running this:
```bash
> spiketag cp sorter-fpga
```

In [ ]:
%matplotlib inline
%gui qt

import numpy as np
import matplotlib.pyplot as plt
from spiketag.mvc import controller
from spiketag.base import probe

mua_filename = './mua.bin'
spk_filename = './spk.bin'
prb_filename = './prb_a2x32.prb'

prb = probe()
prb.load(prb_filename)
ctrl = controller(
                  fpga = True,
                  probe = prb,
                  mua_filename=mua_filename, 
                  spk_filename=spk_filename, 
                  scale=False
                 )

# set threshold
ctrl.set_threshold(beta=4.5)

ref_chNo_l = 38 # change this value
ref_chNo_r = 160 # change this value

if ref_chNo_l != 160:
    ctrl.fpga.thres[ref_chNo_l] = -20000.

if ref_chNo_r != 160:
    ctrl.fpga.thres[ref_chNo_r] = -20000.

ctrl.fpga.thres[128:160] = -20000. # suppress unused channels

In [ ]:
import matplotlib.pyplot as plt

thres = np.zeros(len(prb.chs))
for i, ch in enumerate(prb.chs):
    thres[i] = ctrl.fpga.thres[ch] * 0.195
plt.plot(thres)
thres